In [4]:
# Extract FTSE100 data
#date extrated from (https://uk.finance.yahoo.com/quote/%5EFTSE/) & (https://uk.finance.yahoo.com/crypto/)
#I chose Yahoo finance as it is a public domine and accesable without the need to login 
#and create a personal API key

# Importing library

In [3]:
import yfinance as yf #gets data from yahoo finance public accesable domain 

import pandas as pd #pandas for data manipulation

from datetime import datetime, timedelta #use to convert data into usable date time format

import matplotlib.pyplot as plt #for plotting 

import seaborn as sns# for plotting

### Define the timeframe

In [4]:
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

### Extract FTSE100 data
date extrated from (https://uk.finance.yahoo.com/quote/%5EFTSE/) & (https://uk.finance.yahoo.com/crypto/)I chose Yahoo finance as it is a public domine and accesable without the need to login  and create a personal API key

In [5]:
ftse100 = yf.download('^FTSE', start=start_date, end=end_date)
ftse100.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


### Extract Bitcoin GBP data

In [6]:
bit_gb = yf.download('BTC-GBP', start=start_date, end=end_date)
bit_gb.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [7]:
# Cleaning FTSE100 data
ftse100.dropna(inplace=True)
ftse100['Date'] = pd.to_datetime(ftse100['Date'])

In [8]:
# Cleaning Bitcoin data
bit_gb.dropna(inplace=True)
bit_gb['Date'] = pd.to_datetime(bit_gb['Date'])

In [9]:
# Merge the data on the Date column
data = pd.merge(ftse100, bit_gb[['Date', 'Close']], on='Date', how='inner')
data.rename(columns={'Close_x': 'FTSE100_Close', 'Close_y': 'Bitcoin_Price_GBP'}, inplace=True)

### Calculate Moving Averages for FTSE100

#### The SMA is the arithmetic mean of the closing prices over a specified number of periods.
#### It provides a smoothed line that helps to identify the trend direction.
#### A longer SMA (e.g., 30-day) is smoother and less sensitive to short-term price fluctuations than a shorter SMA (e.g., 7-day)


In [10]:
#calculate 7 and 30 day simple moving averages(SMA)
data['FTSE100_7Day_SMA'] = data['FTSE100_Close'].rolling(window=7).mean()
data['FTSE100_30Day_SMA'] = data['FTSE100_Close'].rolling(window=30).mean()

In [11]:
#Exponential Moving Average (EMA):
#The EMA gives more weight to recent prices, making it more responsive to new information compared to the SMA.
#The span parameter controls how quickly the weights decay for older observations.
#The 7-day EMA responds more quickly to price changes than the 30-day EMA due to its shorter span

In [12]:
#calculate 7 days and 30 days exponential moving average (EMA)
data['FTSE100_7Day_EMA'] =data['FTSE100_Close'].ewm(span=7, adjust=False).mean()
data['FTSE100_30Day_EMA'] =data['FTSE100_Close'].ewm(span=30, adjust=False).mean()

#pandas .ewm() was used as it is a type of moving average to calculate \
#the exponentially weighted moving average for a certain number of previous periods.

#### Exponential Moving Average for BitCoin

In [13]:
#Calculate 20, 50, 200 Exponential Moving Average
bit_gb['20_MA'] = bit_gb['Adj Close'].rolling(window=20).mean()
bit_gb['50_MA'] = bit_gb['Adj Close'].rolling(window=50).mean()
bit_gb['200_MA'] = bit_gb['Adj Close'].rolling(window=200).mean()

## Feature Engineering: Calculating daily returns for FTSE100 and Bitcoin

In [ ]:
# percentage change------.pct_change() calculates the percentage change between 
#the current and previous closing prices

In [14]:
data['FTSE100_Return'] = data['FTSE100_Close'].pct_change()
data['Bitcoin_Return'] = data['Bitcoin_Price_GBP'].pct_change()

In [15]:
# Drop rows with NaN values created by rolling and pct_change
data.dropna(inplace=True)

In [16]:
# Display the first few rows of the merged data
data.head()

,Date,Open,High,Low,FTSE100_Close,Adj Close,Volume,Bitcoin_Price_GBP,FTSE100_7Day_SMA,FTSE100_30Day_SMA,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_Return,Bitcoin_Return
29,2019-08-27,7095.000000,7110.799805,7044.700195,7089.600098,7089.600098,851399100,8292.351562,7135.528669,7347.116699,7133.016819,7297.776151,-0.000761,-0.021773
30,2019-08-28,7089.600098,7131.200195,7050.000000,7114.700195,7114.700195,675638900,7983.849121,7135.171526,7331.700033,7128.437663,7285.964799,0.003540,-0.037203
31,2019-08-29,7114.700195,7199.200195,7103.100098,7184.299805,7184.299805,635681800,7804.203613,7134.400042,7319.993359,7142.403199,7279.405767,0.009783,-0.022501
32,2019-08-30,7184.299805,7240.399902,7179.700195,7207.200195,7207.200195,727868800,7893.874023,7146.142927,7310.463363,7158.602448,7274.747343,0.003188,0.011490
33,2019-09-02,7207.200195,7315.299805,7206.899902,7281.899902,7281.899902,497764900,8575.850586,7157.271484,7302.903353,7189.426811,7275.208798,0.010365,0.086393


In [25]:
# Summary Statistics
print("\nSummary Statistics:\n")
data.describe()


Summary Statistics:



,Open,High,Low,FTSE100_Close,Adj Close,Volume,Bitcoin_Price_GBP,FTSE100_7Day_SMA,FTSE100_30Day_SMA,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_Return,Bitcoin_Return
count,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1.231000e+03,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000
mean,7174.327053,7219.121778,7129.565238,7175.219254,7175.219254,8.435573e+08,23722.513072,7172.605862,7164.175225,7172.633095,7164.678294,0.000185,0.002163
std,628.434904,617.739488,641.207523,629.209957,629.209957,3.220907e+08,13585.263502,622.350882,597.493087,619.883198,588.319457,0.011097,0.040032
min,4993.899902,5181.000000,4898.799805,4993.899902,4993.899902,0.000000e+00,3954.907959,5175.571429,5530.920036,5246.886851,5803.318270,-0.108738,-0.359284
25%,6960.500000,7007.949951,6900.500000,6960.500000,6960.500000,6.428026e+08,11865.009277,6965.628557,6997.670036,6975.792926,6991.180583,-0.004283,-0.016043
50%,7324.500000,7368.299805,7282.500000,7326.100098,7326.100098,7.763260e+08,22547.867188,7325.285645,7316.686637,7328.603601,7313.064306,0.000659,0.001099
75%,7573.300049,7613.400146,7532.300049,7573.600098,7573.600098,9.731642e+08,32963.210938,7564.178571,7529.021647,7567.021528,7532.492527,0.005252,0.019995
max,8445.799805,8474.400391,8427.900391,8445.799805,8445.799805,2.813498e+09,57090.710938,8429.414202,8300.103320,8409.093244,8239.025561,0.090530,0.210290


In [17]:
# Save data as CSV
csv_file_path = 'data.csv'
data.to_csv(csv_file_path, index=False)